In [ ]:
import pandas as pd
df = pd.read_csv('fpl_start.csv')
df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 1246: invalid start byte

In [ ]:
df = pd.read_csv('fpl_start.csv', encoding='ISO-8859-1')
df.head()

In [ ]:
!pip install -q pulp

In [ ]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum
prob = LpProblem('FPL', LpMaximize)
player_vars = LpVariable.dicts('player', df.index, cat='Binary')
team_vars = LpVariable.dicts('team', df['Team'].unique(), cat='Binary')
position_vars = LpVariable.dicts('position', df['Pos'].unique(), cat='Binary')
formation_vars = LpVariable.dicts('formation', ['1-4-4-2', '1-4-3-3', '1-4-5-1', '1-5-4-1', '1-5-3-2', '1-5-2-3', '1-3-5-2', '1-3-4-3'], cat='Binary')

In [ ]:
prob += lpSum([df.loc[i, 'total'] * player_vars[i] for i in df.index]), 'Total Points'

In [ ]:
# Total buying value constraint
prob += lpSum([df.loc[i, 'BV'] * player_vars[i] for i in df.index]) <= 83, 'Total Buying Value'
# Team constraint
for team in df['Team'].unique():
    prob += lpSum([player_vars[i] for i in df[df['Team'] == team].index]) <= 3, f'Team {team}'
# Position and formation constraints
formations = {
    '1-4-4-2': {'G': 1, 'D': 4, 'M': 4, 'F': 2},
    '1-4-3-3': {'G': 1, 'D': 4, 'M': 3, 'F': 3},
    '1-4-5-1': {'G': 1, 'D': 4, 'M': 5, 'F': 1},
    '1-5-4-1': {'G': 1, 'D': 5, 'M': 4, 'F': 1},
    '1-5-3-2': {'G': 1, 'D': 5, 'M': 3, 'F': 2},
    '1-5-2-3': {'G': 1, 'D': 5, 'M': 2, 'F': 3},
    '1-3-5-2': {'G': 1, 'D': 3, 'M': 5, 'F': 2},
    '1-3-4-3': {'G': 1, 'D': 3, 'M': 4, 'F': 3}
}
for formation, positions in formations.items():
    prob += lpSum([position_vars[pos] for pos in positions.keys()]) == lpSum([formation_vars[formation] for formation in formations.keys()]), f'Formation {formation}'
    for pos, count in positions.items():
        prob += lpSum([player_vars[i] for i in df[df['Pos'] == pos].index]) == count * formation_vars[formation], f'Position {pos} in Formation {formation}'

In [ ]:
prob.solve()

print('Optimal Team:')
for i in df.index:
    if player_vars[i].varValue == 1:
        print(df.loc[i, 'Name'])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1b02f82336c04015b10e49a2e9b54cd0-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/1b02f82336c04015b10e49a2e9b54cd0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 66 COLUMNS
At line 8344 RHS
At line 8406 BOUNDS
At line 9054 ENDATA
Problem MODEL has 61 rows, 647 columns and 6478 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 102 fixed, 0 tightened bounds, 0 strengthened rows, 56 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 12 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probi

In [ ]:
from pulp import LpStatus

print('Status:', LpStatus[prob.status])

for v in prob.variables():
    print(v.name, '=', v.varValue)

Status: Optimal
formation_1_3_4_3 = 0.0
formation_1_3_5_2 = 0.0
formation_1_4_3_3 = 0.0
formation_1_4_4_2 = 0.0
formation_1_4_5_1 = 0.0
formation_1_5_2_3 = 0.0
formation_1_5_3_2 = 0.0
formation_1_5_4_1 = 0.0
player_0 = 0.0
player_1 = 0.0
player_10 = 0.0
player_100 = 0.0
player_101 = 0.0
player_102 = 0.0
player_103 = 0.0
player_104 = 0.0
player_105 = 0.0
player_106 = 0.0
player_107 = 0.0
player_108 = 0.0
player_109 = 0.0
player_11 = 0.0
player_110 = 0.0
player_111 = 0.0
player_112 = 0.0
player_113 = 0.0
player_114 = 0.0
player_115 = 0.0
player_116 = 0.0
player_117 = 0.0
player_118 = 0.0
player_119 = 0.0
player_12 = 0.0
player_120 = 0.0
player_121 = 0.0
player_122 = 0.0
player_123 = 0.0
player_124 = 0.0
player_125 = 0.0
player_126 = 0.0
player_127 = 0.0
player_128 = 0.0
player_129 = 0.0
player_13 = 0.0
player_130 = 0.0
player_131 = 0.0
player_132 = 0.0
player_133 = 0.0
player_134 = 0.0
player_135 = 0.0
player_136 = 0.0
player_137 = 0.0
player_138 = 0.0
player_139 = 0.0
player_14 = 0.0
pl

In [ ]:
# Modify the constraints and solve the problem again
prob = LpProblem('FPL', LpMaximize)
player_vars = LpVariable.dicts('player', df.index, cat='Binary')
team_vars = LpVariable.dicts('team', df['Team'].unique(), cat='Binary')
position_vars = LpVariable.dicts('position', df['Pos'].unique(), cat='Binary')
formation_vars = LpVariable.dicts('formation', ['1-4-4-2', '1-4-3-3', '1-4-5-1', '1-5-4-1', '1-5-3-2', '1-5-2-3', '1-3-5-2', '1-3-4-3'], cat='Binary')

# Objective function
prob += lpSum([df.loc[i, 'total'] * player_vars[i] for i in df.index]), 'Total Points'

# Total buying value constraint
prob += lpSum([df.loc[i, 'BV'] * player_vars[i] for i in df.index]) <= 100, 'Total Buying Value'

# Team constraint
for team in df['Team'].unique():
    prob += lpSum([player_vars[i] for i in df[df['Team'] == team].index]) <= 2, f'Team {team}'

# Position and formation constraints
for formation, positions in formations.items():
    prob += lpSum([position_vars[pos] for pos in positions.keys()]) == lpSum([formation_vars[formation] for formation in formations.keys()]), f'Formation {formation}'
    for pos, count in positions.items():
        prob += lpSum([player_vars[i] for i in df[df['Pos'] == pos].index]) == count * formation_vars[formation], f'Position {pos} in Formation {formation}'

# Solve the problem
prob.solve()

# Print the optimal team
print('Optimal Team:')
for i in df.index:
    if player_vars[i].varValue == 1:
        print(df.loc[i, 'Name'])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/dc61d3cc0394492592cafb185e699a15-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/dc61d3cc0394492592cafb185e699a15-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 66 COLUMNS
At line 8344 RHS
At line 8406 BOUNDS
At line 9054 ENDATA
Problem MODEL has 61 rows, 647 columns and 6478 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 109 fixed, 0 tightened bounds, 0 strengthened rows, 56 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 14 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probi

In [ ]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

formations = {
    '1-4-4-2': {'G': 1, 'D': 4, 'M': 4, 'F': 2},
    '1-4-3-3': {'G': 1, 'D': 4, 'M': 3, 'F': 3},
    '1-4-5-1': {'G': 1, 'D': 4, 'M': 5, 'F': 1},
    '1-5-4-1': {'G': 1, 'D': 5, 'M': 4, 'F': 1},
    '1-5-3-2': {'G': 1, 'D': 5, 'M': 3, 'F': 2},
    '1-5-2-3': {'G': 1, 'D': 5, 'M': 2, 'F': 3},
    '1-3-5-2': {'G': 1, 'D': 3, 'M': 5, 'F': 2},
    '1-3-4-3': {'G': 1, 'D': 3, 'M': 4, 'F': 3}
}

optimal_teams = {}

for formation, positions in formations.items():
    prob = LpProblem('FPL', LpMaximize)
    player_vars = LpVariable.dicts('player', df.index, cat='Binary')
    team_vars = LpVariable.dicts('team', df['Team'].unique(), cat='Binary')

    # Objective function
    prob += lpSum([df.loc[i, 'total'] * player_vars[i] for i in df.index]), 'Total Points'

    # Total buying value constraint
    prob += lpSum([df.loc[i, 'BV'] * player_vars[i] for i in df.index]) <= 83, 'Total Buying Value'

    # Team constraint
    for team in df['Team'].unique():
        prob += lpSum([player_vars[i] for i in df[df['Team'] == team].index]) <= 3, f'Team {team}'

    # Position constraints
    for pos, count in positions.items():
        prob += lpSum([player_vars[i] for i in df[df['Pos'] == pos].index]) == count, f'Position {pos}'

    # Solve the problem
    prob.solve()

    # Store the optimal team
    optimal_teams[formation] = [df.loc[i, 'Name'] for i in df.index if player_vars[i].varValue == 1]

# Print the optimal teams
for formation, team in optimal_teams.items():
    print(f'Formation: {formation}')
    print('Team:', team)
    print()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bbee494b53f34c0b9c5e78a91ee343e7-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/bbee494b53f34c0b9c5e78a91ee343e7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3711 RHS
At line 3737 BOUNDS
At line 4373 ENDATA
Problem MODEL has 25 rows, 635 columns and 1905 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 254.644 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 550 columns (550 integer (526 of which binary)) and 1650 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.4
Cbc0038I Pass   1: suminf.    0.10526 (2) obj. -253.511 iterations 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/46f035457690426dbd1dbfc3830fd877-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/46f035457690426dbd1dbfc3830fd877-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3711 RHS
At line 3737 BOUNDS
At line 4373 ENDATA
Problem MODEL has 25 rows, 635 columns and 1905 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 255.978 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 529 columns (529 integer (508 of which binary)) and 1587 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.8
Cbc0038I Pass   1: suminf.    0.21053 (2) obj. -253.724 iterations 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a2d928581e7d469a9266d05dc5560fa1-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/a2d928581e7d469a9266d05dc5560fa1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3711 RHS
At line 3737 BOUNDS
At line 4373 ENDATA
Problem MODEL has 25 rows, 635 columns and 1905 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 250.059 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 550 columns (550 integer (526 of which binary)) and 1650 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.285714
Cbc0038I Solution found of -249.75
Cbc0038I Cleaned solution o

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a9f64770ce1545d4a3060fd0aa64003e-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/a9f64770ce1545d4a3060fd0aa64003e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3711 RHS
At line 3737 BOUNDS
At line 4373 ENDATA
Problem MODEL has 25 rows, 635 columns and 1905 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 256.671 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 25 rows, 550 columns (550 integer (526 of which binary)) and 1650 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.571429
Cbc0038I Pass   1: suminf.    0.21053 (2) obj. -255.152 iterat

In [ ]:
formation_points = {}

for formation, team in optimal_teams.items():
    points = sum(df[df['Name'].isin(team)]['total'])
    formation_points[formation] = points

max_points_formation = max(formation_points, key=formation_points.get)
max_points_team = optimal_teams[max_points_formation]

print(f'Formation with highest total points: {max_points_formation}')
print('Team:', max_points_team)

Formation with highest total points: 1-3-5-2
Team: ['Haaland', 'Saka', 'B.Fernandes', '?degaard', 'Mbeumo', 'Martinelli', 'Watkins', 'Ederson M.', 'Chilwell', 'James', 'Colwill']
